In [1]:
import sys
#sys.path.insert(0,'/home/lshtanko/Programming/another-fem-framework/')
#sys.path.insert(0,'/Users/marusy/Programming/another-fem-framework/')
sys.path.insert(0,'/Users/leonshting/Programming/Schlumberger/fem-framework/')

In [2]:
from operator_assembler.assembly_interface import AssemblyInterface2D
from operator_assembler.n_to_1_ddof_allocator import Nto1Allocator2D
from grid.grid_manager import GridManager
from grid.allocator_interface import InterfaceDofAllocator2D
from grid.grid_domain import GridDomain
from operator_assembler.matrix_assembler import MatrixAssembler2D
from scipy.ndimage import imread
from grid import cell, grid_layer, grid_manager
import h5py
import numpy as np

from interpolant.cell_to_points import *
from common import visual
from common.tests import *
from common.helpers import *
from matplotlib import pyplot as plt
from scipy.sparse import csr_matrix
from scipy.sparse import linalg

%matplotlib inline
plt.rcParams['figure.figsize'] = 10, 7.5

In [3]:
np.set_printoptions(linewidth=500)

#### Bad cell

In [ ]:
data_start = (16, 20)
data_shape = (4, 4)
data_end = tuple([i+j for i,j in zip(data_start, data_shape)])

h5_data = h5py.File('../../../GM_L3D/600/state/viz_GM_1.h5')['index'][0]
data = h5_data[data_start[0]:data_end[0], data_start[1]:data_end[1]]
#data = imread('/Users/marusy/Programming/model/bhi2_labelled0000.tif')[data_start[0]:data_end[0], data_start[1]:data_end[1]]

grid_domain = GridDomain(integer_size=data_shape, domain_size=(1.,1.))
gm = GridManager()
ifma = InterfaceDofAllocator2D(grid_manager=gm.fit(data=data))


gm.draw_grid()

In [4]:
cell15 = cell.Cell2D(size=(1,1), ll_vertex=(0,8))
cell25 = cell.Cell2D(size=(1,1), ll_vertex=(0,9))

cell14 = cell.Cell2D(size=(1,1), ll_vertex=(0,6))
cell24 = cell.Cell2D(size=(1,1), ll_vertex=(0,7))

cell13 = cell.Cell2D(size=(1,1), ll_vertex=(0,4))
cell23 = cell.Cell2D(size=(1,1), ll_vertex=(0,5))

cell12 = cell.Cell2D(size=(1,1), ll_vertex=(0,2))
cell22 = cell.Cell2D(size=(1,1), ll_vertex=(0,3))

cell1 = cell.Cell2D(size=(1,1), ll_vertex=(0,0))
cell2 = cell.Cell2D(size=(1,1), ll_vertex=(0,1))

cell3 = cell.Cell2D(size=(2,2), ll_vertex=(1,0))
cell32 = cell.Cell2D(size=(2,2), ll_vertex=(1,2))
cell33 = cell.Cell2D(size=(2,2), ll_vertex=(1,4))
cell34 = cell.Cell2D(size=(2,2), ll_vertex=(1,6))
cell35 = cell.Cell2D(size=(2,2), ll_vertex=(1,8))

In [5]:
layer1 = grid_layer.GridLayer2D([c.ll_vertex for c in [cell1, cell2, cell12, cell22, cell13, cell23, cell14, cell24, cell15, cell25]], layer_number=0, index=[1,1], div_index=[1,1])
layer2 = grid_layer.GridLayer2D([c.ll_vertex for c in [cell3, cell32, cell33, cell34, cell35]], layer_number=1, index=[1], div_index=[1])

#layer1 = grid_layer.GridLayer2D([c.ll_vertex for c in [cell1, cell2]], layer_number=0, index=[1,1], div_index=[1,1])
#layer2 = grid_layer.GridLayer2D([c.ll_vertex for c in [cell3]], layer_number=1, index=[1], div_index=[1])

In [6]:
gm = grid_manager.GridManager(dim=2, method='coarse2to1')
gm.grid_layers = [layer1, layer2]
ifma = InterfaceDofAllocator2D(grid_manager=gm)

In [7]:
ma = Nto1Allocator2D(grid_interface=ifma)
ma._make_ddof_index()

In [ ]:
#6 cells
ma._vertex_ddof_index[((0, 3), (0, 4))] = (ma._vertex_ddof_index[((0, 3), (0, 4))][0], ma._vertex_ddof_index[((0, 0), (0, 0))][1])
ma._vertex_ddof_index[((0, 3), (1, 4))] = (ma._vertex_ddof_index[((0, 3), (1, 4))][0], ma._vertex_ddof_index[((0, 0), (1, 0))][1])
ma._vertex_ddof_index[((1, 2), (1, 4))] = (ma._vertex_ddof_index[((1, 2), (1, 4))][0], ma._vertex_ddof_index[((1, 0), (1, 0))][1])
ma._vertex_ddof_index[((1, 2), (3, 4))] = (ma._vertex_ddof_index[((1, 2), (3, 4))][0], ma._vertex_ddof_index[((1, 0), (3, 0))][1])

In [ ]:
#3 cells
ma._vertex_ddof_index[((0, 1), (0, 2))] = (ma._vertex_ddof_index[((0, 1), (0, 2))][0], ma._vertex_ddof_index[((0, 0), (0, 0))][1])
ma._vertex_ddof_index[((0, 1), (1, 2))] = (ma._vertex_ddof_index[((0, 1), (1, 2))][0], ma._vertex_ddof_index[((0, 0), (1, 0))][1])
ma._vertex_ddof_index[((1, 0), (1, 2))] = (ma._vertex_ddof_index[((1, 0), (1, 2))][0], ma._vertex_ddof_index[((1, 0), (1, 0))][1])
ma._vertex_ddof_index[((1, 0), (3, 2))] = (ma._vertex_ddof_index[((1, 0), (3, 2))][0], ma._vertex_ddof_index[((1, 0), (3, 0))][1])

In [ ]:
#6 cells
edge_mapping = {((0, 4), (1, 4)): ((0, 0), (1, 0)), ((1, 4), (3, 4)): ((1, 0), (3, 0))}
#3cells
#edge_mapping = {((0, 2), (1, 2)): ((0, 0), (1, 0)), ((1, 2), (3, 2)): ((1, 0), (3, 0))}
for (cell, edge), dd in ma._edge_ddof_index.items():
    if edge_mapping.get(edge) is not None:
        edge_slave = edge
        edge_prim = edge_mapping[edge]
        prim_cell = [i for i in list(ma._edge_ddof_index.keys()) if i[1] == edge_prim][0][0]
        ma._edge_ddof_index[(cell, edge_slave)] = [(i[0],j[1])
            for i,j in
            zip(ma._edge_ddof_index[(cell, edge_slave)], ma._edge_ddof_index[(prim_cell, edge_prim)])
        ]

In [ ]:
#6 cells
ma._weak_edge_connections[ ((0, 2),
  ((1, 2), (1, 3)))] = {((1, 2), ((1, 2), (1, 4))): [(0, 54),
   (1, 74),
   (2, 75),
   (3, 61)]}

ma._weak_edge_connections[((0, 3),
  ((1, 3), (1, 4)))] = {((1, 2), ((1, 2), (1, 4))): [(0, 54),
   (1, 74),
   (2, 75),
   (3, 61)]}

ma._weak_edge_connections[((1, 2),
  ((1, 2), (1, 4)))] = {((0, 2), ((1, 2), (1, 3))): [(12, 19),
   (13, 34),
   (14, 35),
   (15, 31)], ((0, 3), ((1, 3), (1, 4))): [(12, 31),
   (13, 46),
   (14, 47),
   (15, 9)]}

In [ ]:
#3cells
ma._weak_edge_connections[((0, 0), ((1, 0), (1, 1)))] = {((1, 0), ((1, 0), (1, 2))): [(0, 37),
   (1, 38),
   (2, 39),
   (3, 37)]}
ma._weak_edge_connections[((0, 1), ((1, 1), (1, 2)))] = {((1, 0), ((1, 0), (1, 2))): [(0, 37),
   (1, 38),
   (2, 39),
   (3, 37)]}

ma._weak_edge_connections[((1, 0),((1, 0), (1, 2)))] = {((0, 0), ((1, 0), (1, 1))): [(12, 9),
   (13, 10),
   (14, 11),
   (15, 3)], ((0, 1), ((1, 1), (1, 2))): [(12, 3),
   (13, 22),
   (14, 23),
   (15, 9)]}

In [8]:
grid_domain = GridDomain(domain_size=(3.,10.), integer_size=(3,10))
grid_domain.make_pointwise_index(ma)

In [9]:
grid_domain.pointwise_index

{(0.0, 0.0): {6},
 (0.0, 0.27639320225002106): {4},
 (0.0, 0.72360679774997894): {5},
 (0.0, 1.0): {2},
 (0.0, 1.2763932022500211): {20},
 (0.0, 1.7236067977499789): {21},
 (0.0, 2.0): {18},
 (0.0, 2.2763932022500208): {32},
 (0.0, 2.7236067977499792): {33},
 (0.0, 3.0): {30},
 (0.0, 3.2763932022500208): {44},
 (0.0, 3.7236067977499792): {45},
 (0.0, 4.0): {42},
 (0.0, 4.2763932022500208): {56},
 (0.0, 4.7236067977499792): {57},
 (0.0, 5.0): {54},
 (0.0, 5.2763932022500208): {68},
 (0.0, 5.7236067977499792): {69},
 (0.0, 6.0): {66},
 (0.0, 6.2763932022500208): {80},
 (0.0, 6.7236067977499783): {81},
 (0.0, 7.0): {78},
 (0.0, 7.2763932022500217): {92},
 (0.0, 7.7236067977499783): {93},
 (0.0, 8.0): {90},
 (0.0, 8.2763932022500217): {104},
 (0.0, 8.7236067977499783): {105},
 (0.0, 9.0): {102},
 (0.0, 9.2763932022500217): {112},
 (0.0, 9.7236067977499783): {113},
 (0.0, 10.0): {114},
 (0.27639320225002106, 0.0): {7},
 (0.27639320225002106, 0.27639320225002106): {12},
 (0.27639320225002106

In [10]:
ifma2 = AssemblyInterface2D(allocator=ma)
MA = MatrixAssembler2D(assembly_interface=ifma2, grid_domain=grid_domain)

In [11]:
deep = MA.assemble()

[(0, 6), (1, 4), (2, 5), (3, 2), (4, 7), (5, 12), (6, 13), (7, 0), (8, 8), (9, 14), (10, 15), (11, 1), (12, 9), (13, 10), (14, 11), (15, 3)] [(0, 2), (1, 20), (2, 21), (3, 18), (4, 0), (5, 24), (6, 25), (7, 16), (8, 1), (9, 26), (10, 27), (11, 17), (12, 3), (13, 22), (14, 23), (15, 19)]
{0, 1, 2, 4, 5, 6, 7, 8, 12, 13, 14, 15, 16, 17, 18, 20, 21, 24, 25, 26, 27} {128, 129, 130, 131, 132, 136, 137, 138, 139, 124, 125, 127} [[133, 134, 135, 126]] [[9, 10, 11, 3, 22, 23, 19]]
 0[(0, 18), (1, 32), (2, 33), (3, 30), (4, 16), (5, 36), (6, 37), (7, 28), (8, 17), (9, 38), (10, 39), (11, 29), (12, 19), (13, 34), (14, 35), (15, 31)] [(0, 30), (1, 44), (2, 45), (3, 42), (4, 28), (5, 48), (6, 49), (7, 40), (8, 29), (9, 50), (10, 51), (11, 41), (12, 31), (13, 46), (14, 47), (15, 43)]
{16, 17, 18, 28, 29, 30, 32, 33, 36, 37, 38, 39, 40, 41, 42, 44, 45, 48, 49, 50, 51} {140, 141, 143, 144, 145, 148, 149, 150, 151, 124, 125, 127} [[126, 146, 147, 142]] [[19, 34, 35, 31, 46, 47, 43]]
 1[(0, 42), (1, 56

In [ ]:
sine_test = grid_domain.vectorize_function(lambda x,y: np.sin(x+y))
product = grid_domain.devectorize_vector(MA.assembled * sine_test)
init = grid_domain.devectorize_function(lambda x,y: np.sin(x+y))

visual.plot_surface_unstructured_w_dict(
    point_val_dict=grid_domain.devectorize_vector(MA.assembled * sine_test),
    plot_domain_shape=grid_domain.domain_size,
    int_domain_shape=grid_domain.integer_size
)

visual.plot_surface_unstructured_w_dict(
    point_val_dict=grid_domain.devectorize_function(lambda x,y: np.sin(x+y)),
    plot_domain_shape=grid_domain.domain_size,
    int_domain_shape=grid_domain.integer_size
)

In [12]:
def partial_diff_test_w_scale(pointnum, point, matrix, pointdict, mass_matrix=None,
                      multiple_dofs_on_point=True, scale_factor=1, return_what='full'):
    x_der = 0
    xx_der = 0
    yy_der = 0
    y_der = 0
    xy_der = 0

    if not multiple_dofs_on_point:
        for k, v in pointdict.items():
            dx = point[0] - k[0]
            dy = point[1] - k[1]

            x_der += dx * matrix[pointnum, v]
            xx_der += matrix[pointnum, v] * dx ** 2 / 2
            y_der += dy * matrix[pointnum, v]
            yy_der += dy ** 2 / 2 * matrix[pointnum, v]
            xy_der += dx * dy * matrix[pointnum, v]
    else:
        for k, vs in pointdict.items():
            for v in vs:
                dx = point[0] - k[0]
                dy = point[1] - k[1]

                x_der += dx * matrix[pointnum, v]
                xx_der += matrix[pointnum, v] * dx ** 2 / 2
                y_der += dy * matrix[pointnum, v]
                yy_der += dy ** 2 / 2 * matrix[pointnum, v]
                xy_der += dx * dy * matrix[pointnum, v]
                
    if mass_matrix is not None:
        print('Mass coef: {}'.format(mass_matrix[pointnum].sum()))
    print('X derivative: {}'.format(x_der * scale_factor))
    print('Y derivative: {}'.format(y_der * scale_factor))
    print('XX derivative: {}'.format(xx_der * scale_factor))
    print('YY derivative: {}'.format(yy_der * scale_factor))
    print('XY_derivative: {}'.format(xy_der * scale_factor), end='\n\n')
    
    return {'XX': xx_der * scale_factor,
            'YY': yy_der * scale_factor,
            'X': x_der * scale_factor,
            'Y': y_der * scale_factor,           
           }

In [13]:
c2p = Cell2PointsConverter(orders=[[3],[3,3]])

In [14]:
w_ld, w_rd = c2p.get_weights()

In [15]:
w_l, w_r = c2p.get_diag_weights()

In [ ]:
sums = {'XX': 0, 'YY': 0, 'X': 0, 'Y': 0}
for num,i in enumerate([445, 446, 447, 448, 449, 450, 432]):
    print(i, end=' ')
    for k, v in partial_diff_test_w_scale(matrix=MA.half_glob,
                       point=grid_domain.normal_index[i],
                       pointnum=i,
                       pointdict=grid_domain.pointwise_index,
                       scale_factor=1):
        sums[k] += v

In [ ]:
ls = [0.3333333333695328, 0.833333333330938]
w_rd * np.dot(MA.I_s2b, np.array(ls + ls[::-1]))

In [16]:
for i in [31]:
    print(i, end=' ')
    partial_diff_test(matrix=MA.assembled,
                      point=grid_domain.normal_index[i],
                      pointnum=i,
                      pointdict=grid_domain.pointwise_index)

31 X derivative: 8.287339998275245e-13
Y derivative: 2.3193336357377192e-12
XX derivative: -0.020833333334640203
YY derivative: -0.041666096928817536
XY_derivative: -4.270112258578096e-12



In [17]:
for i in [31]:
    print(i, end=' ')
    partial_diff_test_w_scale(matrix=MA.half_glob,
                              point=grid_domain.normal_index[i],
                              pointnum=i,
                              pointdict=grid_domain.pointwise_index,
                              scale_factor=1
                             )

31 X derivative: -0.16666666666871838
Y derivative: 1.8848950178451673e-12
XX derivative: -0.013888888890118753
YY derivative: -0.01388820232166671
XY_derivative: 2.753661021184435e-13



In [ ]:
sums

In [ ]:
plt.imshow(np.vstack((MA.half_glob[158].toarray(),MA.half_glob[159].toarray())), interpolation='none', aspect=10)

In [ ]:
w_rd